In [242]:
import pandas as pd
import numpy as np
import os

In [243]:
folder = r'C:\Users\jcallomamanib\Documents\GitProjects\maintenanceApp\02. Data2HR\data'
asset_data = 'AssetData.csv'
asset_system = 'AssetStructureA&S.csv'
system_component = 'AssetStructureS&C.csv'
component_dutie = 'DutieStructureC&D.csv'
dutie_data = 'DutieStructureD&D.csv'
dutie_resource = 'DutieStructureD&R.csv'
wbs = 'WBS.csv'

ruteA_D = os.path.join(folder,asset_data)
ruteA_S = os.path.join(folder,asset_system)
ruteS_C = os.path.join(folder,system_component)
ruteC_D = os.path.join(folder,component_dutie)
ruteD_D = os.path.join(folder,dutie_data)
ruteD_R = os.path.join(folder,dutie_resource)
rute_wbs = os.path.join(folder,wbs)


In [244]:
dataA_D = pd.read_csv(ruteA_D, dtype = {'Cod sap':str})
dataA_S = pd.read_csv(ruteA_S, dtype = {'System':str})
dataS_C = pd.read_csv(ruteS_C)
dataC_D = pd.read_csv(ruteC_D)
dataD_D = pd.read_csv(ruteD_D, dtype = {'Specialist':str})
dataD_R = pd.read_csv(ruteD_R, dtype = {'Cod. Material':str})
data_wbs = pd.read_csv(rute_wbs)

In [335]:
# All the conditional activities are remove
dataD_D_1task = dataD_D.loc[dataD_D.loc[:,'Maintenance type'] != 'COND',:].reset_index(drop = True)
dataD_D_2task = dataD_D.loc[dataD_D.loc[:,'Maintenance type'] == 'COND',:].reset_index(drop = True)
# All the non Mechanical or Predective activities are remove
dataD_D_1T_MP = dataD_D_1task.loc[(dataD_D_1task.loc[:,'Specialist']=='TEC-MEC') | (dataD_D_1task.loc[:,'Specialist']=='TEC-PRD'),:].reset_index(drop = True)
dataD_D_1T_M = dataD_D_1task.loc[dataD_D_1task.loc[:,'Specialist']=='TEC-MEC',:].reset_index(drop = True)
dataD_D_1T_P = dataD_D_1task.loc[dataD_D_1task.loc[:,'Specialist']=='TEC-PRD',:].reset_index(drop = True)


In [336]:
# Removing Spanish information
asset_SP_EN = ['WBS_1_ES', 'WBS_2_ES', 'WBS_3_ES']
dataA_D_EN = dataA_D.drop(asset_SP_EN, axis=1)
# All Johan&Jorge clearence
dataA_D_JJ = dataA_D_EN.loc[dataA_D.loc[:,'Responsable']=='Johan&Jorge',:].sort_values(by='Asset tag').reset_index(drop = True)

In [339]:
%%time
# 
dataA_D_JJ_S = pd.merge(dataA_D_JJ,dataA_S,how = 'left',on = 'Asset tag')
dataA_S_C_JJ = pd.merge(dataA_D_JJ_S, dataS_C, how='left', on = 'key_system')
dataA_S_C_D_JJ = pd.merge(dataA_S_C_JJ, dataC_D, how='left', on='key_component')
dataAllcompileMP = pd.merge(dataA_S_C_D_JJ, dataD_D_1T_MP, how = 'inner', on = 'key_dutie')
dataAllcompileP = pd.merge(dataA_S_C_D_JJ, dataD_D_1T_P, how = 'inner', on = 'key_dutie')
dataAllcompileM = pd.merge(dataA_S_C_D_JJ, dataD_D_1T_M, how = 'inner', on = 'key_dutie')

Wall time: 51.4 ms


In [466]:
dataAllcompileM.columns

Index(['Asset tag', 'Cod sap', 'WBS_1_EN', 'WBS_2_EN', 'WBS_3_EN',
       'Responsable', 'key_system', 'System', 'key_component', 'Component',
       'key_dutie', 'Primary task', 'Primary task detail activity',
       'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft',
       'Estrategy', 'Unidad Periodo', 'Estrategy_eq'],
      dtype='object')

In [484]:
filtertype = dataAllcompileM.loc[:,'Maintenance type'] == 'INSP'
filterunidad = dataAllcompileM.loc[:,'Unidad Periodo'] == 'H'
filterhour = dataAllcompileM.loc[:,'Estrategy'] == 1400
dataAllcompileM.loc[filtertype & filterunidad,'Estrategy_eq'] = (dataAllcompileM.loc[filtertype & filterunidad,'Estrategy']/(24*7)).round(0)

In [485]:
dataAllcompileM.loc[filtertype & filterunidad,['Estrategy','Estrategy_eq']].groupby(['Estrategy','Estrategy_eq']).size()

Estrategy  Estrategy_eq
165.0      1.0             242
330.0      2.0             101
660.0      4.0             711
1000.0     6.0              24
1400.0     8.0               2
1500.0     9.0               7
2000.0     12.0            168
3000.0     18.0              4
4000.0     24.0             89
6000.0     36.0              1
8000.0     48.0             79
dtype: int64

In [486]:
dataAllcompileM.loc[filtertype & filterunidad & filterhour,['key_dutie','Primary task','Primary task detail activity', 'Constraint', 'Time','Estrategy','Estrategy_eq']]

,key_dutie,Primary task,Primary task detail activity,Constraint,Time,Estrategy,Estrategy_eq
104,2223-ML-201|06-06-01,Inspección de componentes,"Inspección las pantallas del trommel, la espir...",DET,6.0,1400.0,8.0
444,2252-ML-202|05-07-01,Inspección de componentes,"Inspección las pantallas del trommel, la espir...",DET,6.0,1400.0,8.0


In [480]:
dataAllcompileM.loc[filtertype & filterunidad & filterhour,'Primary task detail activity'][307]

'Bloquear el downcomer a intervenir, separar las conexiones laterales de la tubería de aire, revisar el interior de las tubería y limpiar si es necesario, instalar los acople de las líneas de aire, revisar las demás conexiones de aire.'

In [ ]:
dataD_D_1T_M.loc[filtertype & filterunidad,['Estrategy','Estrategy_eq']]

In [397]:
columnfilter = ['key_system', 'Component','Primary task', 'Maintenance type', 'Specialist', 'Constraint', 'Time', 'Amount Craft','Estrategy', 'Unidad Periodo']
# '2253-PU-212'
rowfilter = dataAllcompileM.loc[:,'Asset tag'] == '2221-PU-201'
# dataAllcompile.loc[rowfilter,columnfilter]
dataAllcompileM.loc[rowfilter,columnfilter].groupby(['key_system','Component','Maintenance type','Constraint','Estrategy']).agg({'Primary task': lambda x: "\n".join(x)}).reset_index()

,key_system,Component,Maintenance type,Constraint,Estrategy,Primary task
0,2221-PU-201|01,Acople de alta,INSP,MAR,165.0,Inspección acople de alta de bomba
1,2221-PU-201|01,Acople de baja,INSP,MAR,165.0,Inspección acople de baja de bomba
2,2221-PU-201|01,Reductor de velocidad,INSP,MAR,165.0,Inspección de reductor de bomba\nInspec nivel ...
3,2221-PU-201|01,Reductor de velocidad,PREV,DET,4000.0,Cambio aceite de reductor de bomba
4,2221-PU-201|01,Sistema de enfriamiento,INSP,MAR,660.0,Insp general sistema enfriamiento
5,2221-PU-201|01,Sistema de enfriamiento,PREV,DET,4000.0,Cambio filtros aceite sist enfriam
6,2221-PU-201|01,Sistema de enfriamiento,PREV,DET,8000.0,Cambio bomba aceite sist enfriam
7,2221-PU-201|02,Carcasa,INSP,DET,660.0,Inspección de carcasa de bomba
8,2221-PU-201|02,Carcasa,INSP,MAR,165.0,Inspección stuffing box de carcasa
9,2221-PU-201|02,Carcasa,PREV,DET,1000.0,Cambio spare parte húmeda de bomba\nCambio de ...


In [ ]:
other = dataAllcompileM.loc[:,columnfilter].groupby(['key_system','Component','Maintenance type','Constraint','Estrategy']).size()

In [359]:
other.reset_index().loc[other.reset_index()[0]>1,:]

,key_system,Component,Maintenance type,Constraint,Estrategy,0
2,2221-PU-201|01,Reductor de velocidad,INSP,MAR,165.0,2
9,2221-PU-201|02,Carcasa,PREV,DET,1000.0,4
10,2221-PU-201|02,Carcasa,PREV,DET,8000.0,2
16,2222-CY-201|01,Batería de ciclones,PREV,DET,1000.0,3
17,2222-CY-201|01,Batería de ciclones,PREV,DET,8000.0,4
...,...,...,...,...,...,...
2755,4561-FL-40883|01,Placas,INSP,DET,6.0,2
2758,4561-FL-40883|02,Tanque hidráulico,INSP,MAR,6.0,2
2760,4561-FL-40883|02,Tanque hidráulico,PREV,DET,6.0,2
2765,4561-PU-40850|02,Carcasa,PREV,DET,8000.0,2


In [362]:
dataAllcompileM.loc[rowfilter,columnfilter].reset_index(drop=True)

,key_system,Component,Primary task,Maintenance type,Specialist,Constraint,Time,Amount Craft,Estrategy,Unidad Periodo
0,2221-PU-201|01,Acople de alta,Inspección acople de alta de bomba,INSP,TEC-MEC,MAR,0.25,1,165.0,H
1,2221-PU-201|01,Acople de baja,Inspección acople de baja de bomba,INSP,TEC-MEC,MAR,0.25,1,165.0,H
2,2221-PU-201|01,Reductor de velocidad,Inspección de reductor de bomba,INSP,TEC-MEC,MAR,0.25,1,165.0,H
3,2221-PU-201|01,Reductor de velocidad,Inspec nivel de aceite de reductor,INSP,TEC-MEC,MAR,0.25,1,165.0,H
4,2221-PU-201|01,Reductor de velocidad,Cambio aceite de reductor de bomba,PREV,TEC-MEC,DET,4.00,2,4000.0,H
5,2221-PU-201|01,Sistema de enfriamiento,Insp general sistema enfriamiento,INSP,TEC-MEC,MAR,0.25,1,660.0,H
6,2221-PU-201|01,Sistema de enfriamiento,Cambio bomba aceite sist enfriam,PREV,TEC-MEC,DET,3.00,2,8000.0,H
7,2221-PU-201|01,Sistema de enfriamiento,Cambio filtros aceite sist enfriam,PREV,TEC-MEC,DET,2.00,1,4000.0,H
8,2221-PU-201|02,Carcasa,Inspección de carcasa de bomba,INSP,TEC-MEC,DET,3.00,2,660.0,H
9,2221-PU-201|02,Carcasa,Cambio spare parte húmeda de bomba,PREV,TEC-MEC,DET,12.00,5,1000.0,H
